In [ ]:
#İmportlar
import os
import cv2
import numpy as np
import scipy.io
from joblib import load,dump
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm


In [2]:
#Parametreler
IMAGE_SIZE = (128, 128)
HOG_PIXELS_PER_CELL = (16, 16)
HOG_CELLS_PER_BLOCK = (2, 2)
PCA_COMPONENTS = 100
SVM_KERNEL = 'rbf'
SVM_C = 1.0
SVR_C = 1.0

In [ ]:
#UTKFace için veri yükleme fonksiyonu
#bu kodu chatgpt yardımıyla yazdım.
def load_utkface(folder):
    X, y_age, y_gender = [], [], []
    for fname in os.listdir(folder):
        if not fname.lower().endswith('.jpg'):
            continue
        parts = fname.split('_')
        if len(parts) < 2 or not parts[0].isdigit() or not parts[1].isdigit():
            print(f"Atlandı (geçersiz dosya ismi): {fname}")
            continue
        age = int(parts[0])
        gender = int(parts[1])
        img = cv2.imread(os.path.join(folder, fname))
        if img is None:
            print(f"Atlandı (görüntü yüklenemedi): {fname}")
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, IMAGE_SIZE)
        X.append(img)
        y_age.append(age)
        y_gender.append(gender)
    print(len(X))
    return np.array(X), np.array(y_age), np.array(y_gender)

# Zaten oluşturulduğu için komut satırına aldım

# X, y_age, y_gen = load_utkface("data(utk)")
# X_train, X_test, y_age_train, y_age_test, y_gen_train, y_gen_test = train_test_split(X, y_age, y_gen, test_size=0.2, random_state=42)
# np.save("X_train_utkface.npy", X_train)
# np.save("y_age_train_utkface.npy", y_age_train)
# np.save("y_gender_train_utkface.npy", y_gen_train)
# Test setini kaydet
# np.save("X_test_utkface.npy", X_test)
# np.save("y_age_test_utkface.npy", y_age_test)
# np.save("y_gender_test_utkface.npy", y_gen_test)

In [ ]:
#Mat dosyasından veri okuma
#Bu kodu chatgpt yardımıyla yazdım.
def load_from_mat(mat_path, image_root, max_samples=None, name='imdb'):
    data = scipy.io.loadmat(mat_path)

    image_paths = data[name][0][0][2][0] 
    dob         = data[name][0][0][0][0]
    photo_taken = data[name][0][0][1][0]
    gender      = data[name][0][0][3][0]
    X, y_age, y_gender = [], [], []

    for i in tqdm(range(len(image_paths)), desc=f"Yükleniyor: {name}"):
        if np.isnan(gender[i]) or np.isnan(dob[i]) or np.isnan(photo_taken[i]):
            continue

        gen = int(gender[i])

        age = int(photo_taken[i] - dob[i])

        img_path = os.path.join(image_root, image_paths[i][0])
        if not os.path.exists(img_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue    

        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, IMAGE_SIZE)

        X.append(img)
        y_age.append(age)
        y_gender.append(gen)

        if max_samples and len(X) >= max_samples:
            break

    return X, y_age, y_gender

# IMDB + WIKI birleştirme
def load_imdb_and_wiki(total_samples=20000):
    X1, y_age1, y_gender1 = load_from_mat(
        mat_path='imdb_crop/imdb.mat',
        image_root='imdb_crop',
        name='imdb',
        max_samples=None
    )

    X2, y_age2, y_gender2 = load_from_mat(
        mat_path='wiki_crop/wiki.mat',
        image_root='wiki_crop',
        name='wiki',
        max_samples=None
    )

    # Birleştir
    X_all = np.array(X1 + X2)
    y_age_all = np.array(y_age1 + y_age2)
    y_gender_all = np.array(y_gender1 + y_gender2)

    # Karıştır ve örnekle
    indices = np.random.permutation(len(X_all))[:total_samples]
    X_sub = [X_all[i] for i in indices]
    y_age_sub = [y_age_all[i] for i in indices]
    y_gender_sub = [y_gender_all[i] for i in indices]

    print(f"Alt küme boyutu: {len(X_sub)} (IMDB + WIKI)")

    return X_sub, y_age_sub, y_gender_sub

# Zaten oluşturulduğu için komut satırına aldım
#Eğitim setini kaydet
# X, y_age, y_gen = load_imdb_and_wiki()
# X_train, X_test, y_age_train, y_age_test, y_gen_train, y_gen_test = train_test_split(X, y_age, y_gen, test_size=0.2, random_state=42)
# np.save("X_train_imdbwiki.npy", X_train)
# np.save("y_age_train_imdbwiki.npy", y_age_train)
# np.save("y_gender_train_imdbwiki.npy", y_gen_train)
# Test setini kaydet
# np.save("X_test_imdbwiki.npy", X_test)
# np.save("y_age_test_imdbwiki.npy", y_age_test)
# np.save("y_gender_test_imdbwiki.npy", y_gen_test)


In [5]:
def extract_hog_features(images):
    feats = []
    for img in images:
        h = hog(img,
                pixels_per_cell=HOG_PIXELS_PER_CELL,
                cells_per_block=HOG_CELLS_PER_BLOCK,
                feature_vector=True)
        feats.append(h)
    return np.array(feats)

In [6]:
def hog_and_pca(X, y_age, y_gender, name='dataset'):
    X_hog = extract_hog_features(X)
    pca = PCA(n_components=100)
    X_pca = pca.fit_transform(X_hog)
    dump(pca, f'models/model1_1/pca_{name}.joblib')
    print(f"pca_{name}.joblib kaydedildi.")

In [7]:
def train_and_save_models( X_train_pca, X_test_pca, gen_train, gen_test, age_train, age_test, name='dataset'):

    print(f"\n=== {name.upper()} - SVM / SVR ===")
    svm = SVC(kernel=SVM_KERNEL, C=SVM_C, random_state=42)
    svr = SVR(kernel=SVM_KERNEL, C=SVR_C)

    cv_acc = cross_val_score(svm, X_train_pca, gen_train, cv=5)
    print("SVM Gender CV Accuracy:", np.mean(cv_acc))
    cv_mae = cross_val_score(svr, X_train_pca, age_train, cv=5, scoring='neg_mean_absolute_error')
    print("SVR Age CV MAE:", -np.mean(cv_mae))

    svm.fit(X_train_pca, gen_train)
    svr.fit(X_train_pca, age_train)
    test_acc = svm.score(X_test_pca, gen_test)
    test_mae = np.mean(np.abs(svr.predict(X_test_pca) - age_test))
    print("SVM Gender Test Accuracy:", test_acc)
    print("SVR Age Test MAE:", test_mae)

    dump(svm, f"models/model1_1/svm_gender_{name}.joblib")
    dump(svr, f"models/model1_1/svr_age_{name}.joblib")
    print(f"{name}: SVM & SVR modelleri kaydedildi.")

    print(f"\n=== {name.upper()} - MLP ===")
    mlp_clf = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, random_state=42)
    mlp_reg = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', max_iter=1000, random_state=42)

    cv_acc = cross_val_score(mlp_clf, X_train_pca, gen_train, cv=5)
    print("MLP Gender CV Accuracy:", np.mean(cv_acc))
    cv_mae = cross_val_score(mlp_reg, X_train_pca, age_train, cv=5, scoring='neg_mean_absolute_error')
    print("MLP Age CV MAE:", -np.mean(cv_mae))

    mlp_clf.fit(X_train_pca, gen_train)
    mlp_reg.fit(X_train_pca, age_train)
    test_acc = mlp_clf.score(X_test_pca, gen_test)
    test_mae = np.mean(np.abs(mlp_reg.predict(X_test_pca) - age_test))
    print("MLP Gender Test Accuracy:", test_acc)
    print("MLP Age Test MAE:", test_mae)

    dump(mlp_clf, f"models/model1_1/mlp_gender_{name}.joblib")
    dump(mlp_reg, f"models/model1_1/mlp_age_{name}.joblib")
    print(f"{name}: MLP modelleri kaydedildi.")

    print(f"\n=== {name.upper()} - RANDOM FOREST ===")
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)

    cv_acc = cross_val_score(rf_clf, X_train_pca, gen_train, cv=5)
    print("RF Gender CV Accuracy:", np.mean(cv_acc))
    cv_mae = cross_val_score(rf_reg, X_train_pca, age_train, cv=5, scoring='neg_mean_absolute_error')
    print("RF Age CV MAE:", -np.mean(cv_mae))

    rf_clf.fit(X_train_pca, gen_train)
    rf_reg.fit(X_train_pca, age_train)
    test_acc = rf_clf.score(X_test_pca, gen_test)
    test_mae = np.mean(np.abs(rf_reg.predict(X_test_pca) - age_test))
    print("RF Gender Test Accuracy:", test_acc)
    print("RF Age Test MAE:", test_mae)

    dump(rf_clf, f"models/model1_1/rf_gender_{name}.joblib")
    dump(rf_reg, f"models/model1_1/rf_age_{name}.joblib")
    print(f"{name}: Random Forest modelleri kaydedildi.")

In [8]:
def test_model(X_test, y_age_test, y_gender_test, name='dataset'):
    model_dir = f'models/model1_1/'

    # Model ve PCA'yı yükle
    pca = load(f'{model_dir}pca_{name}.joblib')
    X_hog = extract_hog_features(X_test)
    X_pca = pca.transform(X_hog)

    print(f"\n=== [ {name.upper()} ] TEST SONUÇLARI ===")

    # --- SVM + SVR ---
    svm = load(f'{model_dir}svm_gender_{name}.joblib')
    svr = load(f'{model_dir}svr_age_{name}.joblib')
    acc_svm = svm.score(X_pca, y_gender_test)
    mae_svr = mean_absolute_error(y_age_test, svr.predict(X_pca))
    print(f"[SVM] Gender Accuracy: {acc_svm:.3f}")
    print(f"[SVR] Age MAE: {mae_svr:.2f}")

    # --- MLP ---
    mlp_clf = load(f'{model_dir}mlp_gender_{name}.joblib')
    mlp_reg = load(f'{model_dir}mlp_age_{name}.joblib')
    acc_mlp = mlp_clf.score(X_pca, y_gender_test)
    mae_mlp = mean_absolute_error(y_age_test, mlp_reg.predict(X_pca))
    print(f"[MLP] Gender Accuracy: {acc_mlp:.3f}")
    print(f"[MLP] Age MAE: {mae_mlp:.2f}")

    # --- Random Forest ---
    rf_clf = load(f'{model_dir}rf_gender_{name}.joblib')
    rf_reg = load(f'{model_dir}rf_age_{name}.joblib')
    acc_rf = rf_clf.score(X_pca, y_gender_test)
    mae_rf = mean_absolute_error(y_age_test, rf_reg.predict(X_pca))
    print(f"[RF]  Gender Accuracy: {acc_rf:.3f}")
    print(f"[RF]  Age MAE: {mae_rf:.2f}")

In [9]:
# Eğitim verisi
X_train = np.load("X_train_utkface.npy", allow_pickle=True)
y_age_train = np.load("y_age_train_utkface.npy")
y_gen_train = np.load("y_gender_train_utkface.npy")

# Test verisi
X_test = np.load("X_test_utkface.npy", allow_pickle=True)
y_age_test = np.load("y_age_test_utkface.npy")
y_gen_test = np.load("y_gender_test_utkface.npy")

In [10]:
hog_and_pca(X_train, y_age_train, y_gen_train, name="UTKface_1")
pca = load("models/model1_1/pca_UTKface_1.joblib")

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

X_train_pca = pca.transform(X_train_hog)
X_test_pca = pca.transform(X_test_hog)

train_and_save_models(X_train_pca, X_test_pca, y_gen_train, y_gen_test, y_age_train, y_age_test, name="UTKface_1")

pca_UTKface_1.joblib kaydedildi.

=== UTKFACE_1 - SVM / SVR ===


c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


SVM Gender CV Accuracy: 0.7529408455427327
SVR Age CV MAE: 13.797019645843932
SVM Gender Test Accuracy: 0.7574548509029819
SVR Age Test MAE: 13.631828385001866
UTKface_1: SVM & SVR modelleri kaydedildi.

=== UTKFACE_1 - MLP ===


c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MLP Gender CV Accuracy: 0.7104598711366931


c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the 

MLP Age CV MAE: 17.948794555995327


c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


MLP Gender Test Accuracy: 0.7162956740865183
MLP Age Test MAE: 16.824315253816067
UTKface_1: MLP modelleri kaydedildi.

=== UTKFACE_1 - RANDOM FOREST ===
RF Gender CV Accuracy: 0.6865675556124261
RF Age CV MAE: 14.476848103024466
RF Gender Test Accuracy: 0.6955060898782024
RF Age Test MAE: 14.234029137826335
UTKface_1: Random Forest modelleri kaydedildi.


In [11]:
test_model(X_test, y_age_test, y_gen_test, name="UTKface_1")


=== [ UTKFACE_1 ] TEST SONUÇLARI ===
[SVM] Gender Accuracy: 0.757
[SVR] Age MAE: 13.63
[MLP] Gender Accuracy: 0.716
[MLP] Age MAE: 16.82
[RF]  Gender Accuracy: 0.696
[RF]  Age MAE: 14.23


In [ ]:
# Eğitim verisi
X_train = np.load("X_train_imdbwiki.npy", allow_pickle=True)
y_age_train = np.load("y_age_train_imdbwiki.npy")
y_gen_train = np.load("y_gender_train_imdbwiki.npy")

# Test verisi
X_test = np.load("X_test_imdbwiki.npy", allow_pickle=True)
y_age_test = np.load("y_age_test_imdbwiki.npy")
y_gen_test = np.load("y_gender_test_imdbwiki.npy")

In [13]:
hog_and_pca(X_train, y_age_train, y_gen_train, name="imdb_wiki_1")
pca = load("models/model1_1/pca_imdb_wiki_1.joblib")

X_train_hog = extract_hog_features(X_train)
X_test_hog = extract_hog_features(X_test)

X_train_pca = pca.transform(X_train_hog)
X_test_pca = pca.transform(X_test_hog)

train_and_save_models(X_train_pca, X_test_pca, y_gen_train, y_gen_test, y_age_train, y_age_test, name="imdb_wiki_1")

pca_imdb_wiki_1.joblib kaydedildi.

=== IMDB_WIKI_1 - SVM / SVR ===
SVM Gender CV Accuracy: 0.72725
SVR Age CV MAE: 4662.435734554407
SVM Gender Test Accuracy: 0.74175
SVR Age Test MAE: 4795.827861486058
imdb_wiki_1: SVM & SVR modelleri kaydedildi.

=== IMDB_WIKI_1 - MLP ===
MLP Gender CV Accuracy: 0.6445624999999999


c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the 

MLP Age CV MAE: 185886.4781479259


c:\Users\apayd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLP Gender Test Accuracy: 0.65075
MLP Age Test MAE: 36952.376708287506
imdb_wiki_1: MLP modelleri kaydedildi.

=== IMDB_WIKI_1 - RANDOM FOREST ===
RF Gender CV Accuracy: 0.6873750000000001
RF Age CV MAE: 4799.259178978527
RF Gender Test Accuracy: 0.69475
RF Age Test MAE: 4880.149063410822
imdb_wiki_1: Random Forest modelleri kaydedildi.


In [14]:
test_model(X_test, y_age_test, y_gen_test, name="imdb_wiki_1")


=== [ IMDB_WIKI_1 ] TEST SONUÇLARI ===
[SVM] Gender Accuracy: 0.742
[SVR] Age MAE: 4795.83
[MLP] Gender Accuracy: 0.651
[MLP] Age MAE: 36952.38
[RF]  Gender Accuracy: 0.695
[RF]  Age MAE: 4880.15
